In [1]:
!pip install torch torchvision torchaudio

In [1]:
from sample import BaseModel, train_predictor, train_selector
from sample import PredictorNetwork
from sample import SelectorNetwork
import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load the pretrained ResNet-50 model
resnet = models.resnet18(pretrained=True)
selector = SelectorNetwork(32000)
predictor = PredictorNetwork(200704, 32000)
resnet.fc = nn.Linear(resnet.fc.in_features, 10)

p_model_path = "models/p_layer1_v1.pth"
s_model_path = "models/s_layer1_v1.pth"
# Switch to evaluation mode

selector.load_state_dict(torch.load(s_model_path))
predictor.load_state_dict(torch.load(p_model_path))

resnet.eval()
selector.eval()
predictor.eval()
# Transformation and data loading
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Define your input data (example input tensor)
# Load the validation dataset
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Iterate through each layer in the ResNet-50 model and apply them sequentially
total_start = time.time()

for image, labels in val_loader:
  output = image
  for name, layer in resnet.named_children():
      start = time.time()
      if name == 'avgpool':
          output = nn.functional.adaptive_avg_pool2d(output, (1, 1))
      elif name == 'fc':
          output = output.view(output.size(0), -1)
      elif name == 'layer1':# or name == 'layer2' or name == 'layer3' or name == 'layer4':
          output = layer(output)
          print(f"layer1 output shape = {output.shape}")
          pred_out = predictor(output)
          if selector(pred_out) == 1:
              output = pred_out
              break    
      else:
          output = layer(output)

      print(f"Layer: {name}, Output shape: {output.shape}, total time: {time.time() - start}")

print(f"total time: {time.time() - total_start}")


cpu


/Users/erik-luna/miniforge3/envs/cs221_hw7/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/erik-luna/miniforge3/envs/cs221_hw7/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 